In [42]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import models
from keras import layers
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt
import scikitplot as skplot
from SwarmPackagePy import cso

In [2]:
df = pd.read_csv('../Datasets/Final_Dataset.csv', index_col=[0])

In [3]:
df.head()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,gender,jaundice,PDD,Class/ASD,age_range_'12-16 years',age_range_'18 and more',age_range_'4-11 years'
0,1,1,1,1,0,0,1,1,0,0,0,0,0,0,0,1,0
1,1,1,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0
2,1,1,0,1,1,0,1,1,1,1,1,1,1,1,0,1,0
3,1,1,0,1,0,0,1,1,0,1,0,0,1,0,0,1,0
4,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [4]:
df.shape

(1100, 17)

In [5]:
labels = df.pop('Class/ASD')
y = np.array(labels)

In [6]:
X = df.to_numpy()

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, shuffle=True)

In [8]:
print(f'We have {X_train.shape[0]} train samples')
print(f'We have {X_test.shape[0]} test samples')

We have 880 train samples
We have 220 test samples


In [25]:
network = models.Sequential()
network.add(layers.Dense(units=32, activation='sigmoid', input_shape=(X_train.shape[1],)))
network.add(layers.Dense(units=32, activation='sigmoid'))
network.add(layers.Dense(units=1, activation='sigmoid'))
# network.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy']) 

In [26]:
network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 1,633
Trainable params: 1,633
Non-trainable params: 0
_________________________________________________________________


In [27]:
def minimization_function(x):
#     print(len(x))
    layer1_weights = np.array(x[0:512]).reshape(16,32)
    layer1_biases = np.array(x[512:544])
    
    layer2_weights = np.array(x[544:1568]).reshape(32,32)
    layer2_biases = np.array(x[1568:1600])
    
    layer3_weights = np.array(x[1600:1632]).reshape(32,1)
    layer3_biases = np.array(x[1632:1633])
    
    network.layers[0].set_weights([layer1_weights, layer1_biases])
    network.layers[1].set_weights([layer2_weights, layer2_biases])
    network.layers[2].set_weights([layer3_weights, layer3_biases])
    
    cso_predict = network.predict(X_train)    
    return mean_squared_error(y_train, cso_predict)

In [35]:
swarm = cso(150, minimization_function, dimension=1633, iteration=100, pa=0.25, nest=100, lb=-10, ub=10)

In [36]:
best_solution = swarm.get_Gbest()

In [37]:
layer1_weights = np.array(best_solution[0:512]).reshape(16,32)
layer1_biases = np.array(best_solution[512:544])

layer2_weights = np.array(best_solution[544:1568]).reshape(32,32)
layer2_biases = np.array(best_solution[1568:1600])

layer3_weights = np.array(best_solution[1600:1632]).reshape(32,1)
layer3_biases = np.array(best_solution[1632:1633])

network.layers[0].set_weights([layer1_weights, layer1_biases])
network.layers[1].set_weights([layer2_weights, layer2_biases])
network.layers[2].set_weights([layer3_weights, layer3_biases])

In [38]:
predictions = network.predict(X_test)

In [39]:
predictions[predictions > 0.5] = 1
predictions[predictions < 0.5] = 0

In [43]:
print("--------------------- RESULTS OF CUCKOO SWARM OPTIMIZATION -----------------------")
print("------------------------------------------------------------------------")
print("Accuracy Score: ", accuracy_score(y_test, predictions))
print("------------------------------------------------------------------------")
print("Mean Squared Error: ", mean_squared_error(y_test, predictions))
print("------------------------------------------------------------------------")
print("Confusion Matrix: \n\n",confusion_matrix(y_test, predictions))
print("------------------------------------------------------------------------")
print("Classification Report: \n\n",classification_report(y_test, predictions))
print("------------------------------------------------------------------------")

--------------------- RESULTS OF CUCKOO SWARM OPTIMIZATION -----------------------
------------------------------------------------------------------------
Accuracy Score:  0.8727272727272727
------------------------------------------------------------------------
Mean Squared Error:  0.12727272727272726
------------------------------------------------------------------------
Confusion Matrix: 

 [[131  12]
 [ 16  61]]
------------------------------------------------------------------------
Classification Report: 

               precision    recall  f1-score   support

           0       0.89      0.92      0.90       143
           1       0.84      0.79      0.81        77

    accuracy                           0.87       220
   macro avg       0.86      0.85      0.86       220
weighted avg       0.87      0.87      0.87       220

------------------------------------------------------------------------
